In [9]:
import os
import pandas as pd
import pickle
import altair as alt

In [10]:
weather_path1 = "fids_data2.pkl"
weather_path2 = "fids_data3.pkl"
weather_path3 = "output20v2.csv"

In [11]:
weather_data1 = pickle.load(open(weather_path1,'rb'))
weather_data2 = pickle.load(open(weather_path2,'rb'))
weather_data3 = pd.read_csv(weather_path3)

In [12]:
weather_data3

,Unnamed: 0,date,datatype,station,attributes,value,locationid,state
0,0,2020-01-01T00:00:00,TAVG,GHCND:USR0000ABAN,",,U,",72,FIPS:01,Alabama
1,1,2020-01-01T00:00:00,TAVG,GHCND:USR0000ABNS,",,U,",135,FIPS:01,Alabama
2,2,2020-01-01T00:00:00,TAVG,GHCND:USR0000ALIR,",,U,",38,FIPS:01,Alabama
3,3,2020-01-01T00:00:00,TAVG,GHCND:USR0000AOKM,",,U,",66,FIPS:01,Alabama
4,4,2020-01-01T00:00:00,TAVG,GHCND:USR0000AOPE,",,U,",102,FIPS:01,Alabama
...,...,...,...,...,...,...,...,...
854417,169,2020-06-18T00:00:00,TAVG,GHCND:WQW00041606,"H,,S,",289,FIPS:79,Wake Island
854418,170,2020-06-19T00:00:00,TAVG,GHCND:WQW00041606,"H,,S,",281,FIPS:79,Wake Island
854419,171,2020-06-20T00:00:00,TAVG,GHCND:WQW00041606,"H,,S,",289,FIPS:79,Wake Island
854420,172,2020-06-21T00:00:00,TAVG,GHCND:WQW00041606,"H,,S,",287,FIPS:79,Wake Island


In [13]:
def get_locations_in_country(df,country,fids_dict):
    not_in = ["American Samoa",
            "Chicago",
            "Diamond Princess cruise ship",
            "Grand Princess",
            "Grand Princess Cruise Ship",
            "Guam",
            "Northern Mariana Islands",
            "Puerto Rico",
            "Recovered",
            "Unassigned Location (From Diamond Princess)",
            "United States Virgin Islands",
            "US",
            "Virgin Islands",
            "Wuhan Evacuee"]
    locations = df[(df['Country/Region']==country)]['Province/State'].unique()
    locations = filter(lambda location:"," not in location,locations)
    locations = [location for location in locations if location not in not_in]
    fids = list(map(fids_dict.get,locations))
    return [f for f in fids if f is not None],dict(zip(fids,locations))
covid_df_head = './covid_data'
covid_dfs = dict(map(lambda p:(os.path.splitext(p)[0],pd.read_csv(os.path.join(covid_df_head,p))),os.listdir(covid_df_head)))
covid_dfs['covid_19_data'][covid_dfs['covid_19_data']['Country/Region']=='US']
fids_dict = {v['name']:v['id'] for k,v in pd.read_csv('./ncei_data/locations.txt').to_dict('index').items()}
fids_locations,location_dict = get_locations_in_country(covid_dfs['covid_19_data'],'US',fids_dict)

In [14]:
print(weather_data1['FIPS:48']['datatype'].unique())
print(weather_data2['FIPS:48']['datatype'].unique())

['PRCP' 'SNOW' 'SNWD' 'DAPR' 'MDPR' 'WESD' 'WESF']
['PRCP' 'SNOW' 'DAPR' 'MDPR' 'SNWD' 'WESD' 'WESF']


In [150]:
def convert_to_fahrenheit(t,factor=10):
    return (t/factor)*(9./5) + 32

def clean_weather_dataframe(df,fids,location,c_to_f=False):
    df = df[(df['datatype']=='TAVG')]
    if c_to_f:
        df['value'] = df['value'].apply(convert_to_fahrenheit)

    df = df.groupby(['date']).mean()
    #df = df.reset_index().pivot(index='date',columns=['datatype'],values='value')
    df = df.drop([c for c in df.columns if c !='value'],axis=1)
    df = pd.DataFrame(df).rename(columns={'value':'Average Temperature (deg F)'})
    
    return df

In [151]:
# ex_fids = list(weather_data.keys())[0]
# ex_fids_data = clean_weather_dataframe(weather_data[ex_fids],ex_fids,location_dict[ex_fids]).reset_index()
# alt.Chart(ex_fids_data).mark_point().encode(
#     y='Average Temperature (deg F)',
#     x='date'
# )



In [152]:
fids_timeseries = {}
wd3_locs = map(fids_dict.get,weather_data3['state'].unique())
#print(weather_data.keys())
for fids in wd3_locs:#weather_data3:
    if fids in location_dict:
        if fids in weather_data1:
        #print(fids,location_dict[fids])
            fids_timeseries[location_dict[fids]] = clean_weather_dataframe(weather_data1[fids],fids,location_dict[fids])
        if fids in weather_data2:
            if location_dict[fids] in fids_timeseries:
                fids_timeseries[location_dict[fids]] = pd.concat([fids_timeseries[location_dict[fids]],
                                                                              clean_weather_dataframe(weather_data2[fids],fids,location_dict[fids])])
            else:
                fids_timeseries[location_dict[fids]] = clean_weather_dataframe(weather_data2[fids],fids,location_dict[fids])
        wenjun_data = clean_weather_dataframe(weather_data3[weather_data3['state']==location_dict[fids]],fids,location_dict[fids],c_to_f=True)
        #fids_timeseries[location_dict[fids]].update(wenjun_data) #= pd.concat(fids_timeseries[location_dict[fids]],wenjun_data[~wenjun_data.])

        fids_timeseries[location_dict[fids]]['location']=location_dict[fids]
        wenjun_data['location']=location_dict[fids]

        to_update = wenjun_data[(~wenjun_data['location'].isin(fids_timeseries[location_dict[fids]]['location'])
                                )|(~wenjun_data.index.isin(fids_timeseries[location_dict[fids]].index))]
        if len(fids_timeseries[location_dict[fids]])!=0:
            #print(location_dict[fids])#,fids_timeseries[location_dict[fids]])
            fids_timeseries[location_dict[fids]].update(wenjun_data,overwrite=False)
        else:
            fids_timeseries[location_dict[fids]] = wenjun_data
        fids_timeseries[location_dict[fids]]['fids']=fids
print(len(fids_timeseries))

50


In [153]:
print(fids_timeseries['Texas'])
loc_timeseries = pd.concat([df for df in fids_timeseries.values()])

                     Average Temperature (deg F) location     fids
date                                                              
2020-01-01T00:00:00                    49.559730    Texas  FIPS:48
2020-01-02T00:00:00                    54.796757    Texas  FIPS:48
2020-01-03T00:00:00                    52.719459    Texas  FIPS:48
2020-01-04T00:00:00                    49.190000    Texas  FIPS:48
2020-01-05T00:00:00                    53.726486    Texas  FIPS:48
...                                          ...      ...      ...
2020-12-27T00:00:00                    58.210959    Texas  FIPS:48
2020-12-28T00:00:00                    57.836164    Texas  FIPS:48
2020-12-29T00:00:00                    61.485479    Texas  FIPS:48
2020-12-30T00:00:00                    56.263014    Texas  FIPS:48
2020-12-31T00:00:00                    43.909589    Texas  FIPS:48

[366 rows x 3 columns]


In [154]:
loc_timeseries

,Average Temperature (deg F),location,fids
date,,,
2020-02-01T00:00:00,46.923077,Alabama,FIPS:01
2020-02-02T00:00:00,48.923077,Alabama,FIPS:01
2020-02-03T00:00:00,57.076923,Alabama,FIPS:01
2020-02-04T00:00:00,62.538462,Alabama,FIPS:01
2020-02-05T00:00:00,65.692308,Alabama,FIPS:01
...,...,...,...
2020-12-25T00:00:00,27.888889,Wyoming,FIPS:56
2020-12-26T00:00:00,29.411765,Wyoming,FIPS:56
2020-12-27T00:00:00,21.461538,Wyoming,FIPS:56


In [101]:
loc_timeseries.describe()

,Average Temperature (deg F)
count,14929.000000
mean,53.972169
std,17.539477
min,-13.304688
25%,40.500000
50%,55.281250
75%,68.833333
max,88.680541


In [102]:
#the max value doesn't seem correct for Texas in June.. I think we lost some points
set(location_dict.values())-set(loc_timeseries['location'].unique())

{'District of Columbia'}

In [103]:
#We lost some pretty big data points - California - Illinois - New York - Texas. The 4 states with the four biggest populations
charts = [alt.Chart(loc_timeseries[loc_timeseries['location']==loc].reset_index()
                   ).mark_circle().encode(x='date:T',y='Average Temperature (deg F)') for loc in loc_timeseries['location'].unique()]
alt.layer(*charts).properties(width=500)

alt.LayerChart(...)

In [104]:
alt.data_transformers.disable_max_rows()
scatter = alt.Chart(loc_timeseries.reset_index()).mark_point().encode(x='date:T',y='mean(Average Temperature (deg F))')
#print(loc_timeseries.reset_index().groupby('date').mean().reset_index())
mean = alt.Chart(loc_timeseries.reset_index().groupby('date').mean().reset_index()).transform_window(rolling_mean='mean(Average Temperature (deg F))',frame=[-7,7]
                                                               ).mark_line(color='red',size=5).encode(
                                                                    x='date:T',
                                                                    y=alt.Y('rolling_mean:Q'))#,scale=alt.Scale(domain=[20,80])))

(scatter+mean).properties(width=400)

alt.LayerChart(...)

In [105]:
loc_timeseries.to_csv('weather_timeseries.csv')